In [1]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

# Build paths
source_path = Path(DOWNLOAD_DIR) / f'df_finviz_{date_str}.pkl'
dest_path = Path(DEST_DIR) / 'df_finviz.pkl'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

# Example file operations:
# import pandas as pd
# df = pd.read_pickle(source_path)
# df.to_pickle(dest_path)

source_path: C:\Users\ping\Downloads\df_finviz_2025-03-31.pkl
dest_path: ..\data\df_finviz.pkl


In [2]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
# pd.set_option('display.width', None)        # Let the display adjust to the window
# pd.set_option('display.max_colwidth', None) # Show full content of each cell

In [3]:
df = pd.read_pickle(source_path)
display(df.info(), df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   No.              1560 non-null   object
 1   Ticker           1560 non-null   object
 2   Company          1560 non-null   object
 3   Sector           1560 non-null   object
 4   Industry         1560 non-null   object
 5   Market Cap       1560 non-null   object
 6   Dividend         1560 non-null   object
 7   Perf Week        1560 non-null   object
 8   Perf Month       1560 non-null   object
 9   Perf Quart       1560 non-null   object
 10  Perf Half        1560 non-null   object
 11  Perf Year        1560 non-null   object
 12  Perf YTD         1560 non-null   object
 13  Beta             1560 non-null   object
 14  ATR              1560 non-null   object
 15  Volatility W     1560 non-null   object
 16  Volatility M     1560 non-null   object
 17  SMA20            1560 non-null   

None

,No.,Ticker,Company,Sector,Industry,Market Cap,Dividend,Perf Week,Perf Month,Perf Quart,Perf Half,Perf Year,Perf YTD,Beta,ATR,Volatility W,Volatility M,SMA20,SMA50,SMA200,50D High,50D Low,52W High,52W Low,All-Time High,All-Time Low,RSI,Gap,Avg Volume,Rel Volume,Volume,Price,Change,Single Category,Asset Type,AUM
0,1081,CE,Celanese Corp,Basic Materials,Chemicals,6.34B,0.23%,-4.49%,11.44%,-17.90%,-59.40%,-66.54%,-17.97%,1.05,2.59,3.84%,4.74%,1.16%,-7.52%,-43.17%,-25.15%,23.57%,-67.01%,23.57%,-67.84%,894.22%,46.84,-1.01%,3.41M,0.67,"2,271,911",56.77,-0.75%,-,-,-
1,1082,JBTM,JBT Marel Corp,Industrials,Specialty Industrial Machinery,6.34B,0.33%,-4.70%,-7.42%,-7.19%,25.57%,16.87%,-3.86%,1.13,4.08,3.13%,3.18%,-3.37%,-3.81%,10.78%,-12.12%,4.77%,-12.12%,47.87%,-31.18%,1988.89%,42.17,-0.80%,596.93K,0.68,"404,715",122.20,-0.03%,-,-,-
2,1083,BRKR,Bruker Corp,Healthcare,Medical Devices,6.33B,0.48%,-6.20%,-11.61%,-28.42%,-40.74%,-55.53%,-28.80%,1.18,1.79,3.62%,3.83%,-7.96%,-17.14%,-29.39%,-31.58%,-0.14%,-55.76%,-0.14%,-56.00%,1511.58%,27.14,-1.09%,1.76M,0.91,"1,603,346",41.74,-0.81%,-,-,-
3,1084,LTH,Life Time Group Holdings Inc,Consumer Cyclical,Leisure,6.33B,-,-6.99%,-0.85%,36.04%,23.67%,95.98%,36.53%,1.85,1.23,3.79%,4.20%,-0.41%,-0.87%,21.96%,-10.23%,22.19%,-10.23%,144.73%,-10.23%,245.14%,48.06,-2.56%,2.30M,1.46,"3,364,044",30.20,-0.98%,-,-,-
4,1085,PSN,Parsons Corp,Technology,Information Technology Services,6.32B,-,-2.07%,1.70%,-36.90%,-42.85%,-28.39%,-35.82%,0.66,2.21,2.41%,3.74%,0.15%,-13.34%,-32.56%,-39.53%,8.52%,-48.37%,8.52%,-48.37%,140.01%,39.95,-1.40%,1.49M,0.57,"851,080",59.21,-0.35%,-,-,-


In [4]:
import numpy as np

def convert_B_M_K_to_million(value_str):
  """
  Convert financial string values with suffixes to numeric values in billions
  Examples:
  '104.27B' -> 104,270
  '104.27M' -> 104.27
  '104.27K' -> 0.10427
  '-' -> np.nan
  """
  
  if not isinstance(value_str, str):
    return value_str  # Return the original value if not a string
    
  if value_str == '-':
    return np.nan
    
  # Remove any whitespace
  value_str = value_str.strip()
  
  # Handle suffixes
  multipliers = {
    'B': 1000,  
    'M': 1,
    'K': 0.001
  }
  
  try:
    # Extract the numeric part and suffix
    if value_str[-1] in multipliers:
      number = float(value_str[:-1])
      multiplier = multipliers[value_str[-1]]
      return number * multiplier
    else:
      # If no suffix, try to convert directly to float
      return float(value_str)
  except (ValueError, IndexError):
    return np.nan

# Example usage:
# df['Market Cap Numeric'] = df['Market Cap'].apply(convert_B_M_K_to_million)

In [5]:
import pandas as pd
import numpy as np

def process_percentage_columns(df):
    """
    Identifies and processes columns in a DataFrame where values end with '%'.
    The function cleans, converts to numeric, renames, and prints which columns were modified.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The modified DataFrame with processed percentage columns.
                      Returns the original DataFrame if no percentage columns are found.
    """

    # Identify columns where values END WITH '%'
    percent_cols = [
        col for col in df.columns
        if df[col].dtype == 'object'
        and df[col].str.strip().str.endswith('%', na=False).any()
    ]

    # If no percentage columns are found, return the original DataFrame
    if not percent_cols:
        print("No percentage columns found to modify.")  # Print message
        return df

    print("The following columns ending with % were modified:") #Print message before the loop

    # Process identified percentage columns
    for col in percent_cols:
        # Clean data: (1) Strip whitespace, (2) Handle '-', (3) Remove trailing %
        cleaned_series = (
            df[col].str.strip()
            .replace('-', np.nan)  # Convert '-' to NaN
            .str.replace(r'%$', '', regex=True)  # Remove only ENDING %
        )
        
        # Convert to numeric (coerce invalid values to NaN)
        df[col] = pd.to_numeric(cleaned_series, errors='coerce')
        
        # Rename column
        df.rename(columns={col: f"{col} %"}, inplace=True)
        print(f"- {col}")  # Print the column name
        
    return df

# Example Usage:
# Assuming you have a DataFrame called 'df'
# df = process_percentage_columns(df)

In [6]:
df = process_percentage_columns(df)

The following columns ending with % were modified:
- Dividend
- Perf Week
- Perf Month
- Perf Quart
- Perf Half
- Perf Year
- Perf YTD
- Volatility W
- Volatility M
- SMA20
- SMA50
- SMA200
- 50D High
- 50D Low
- 52W High
- 52W Low
- All-Time High
- All-Time Low
- Gap
- Change


In [7]:
# List of columns to concatenate
columns_to_concat = ["Sector", "Industry", "Single Category", "Asset Type"]

# Replace '-' with empty string in specified columns
for col in columns_to_concat:
    df[col] = df[col].replace('-', '')

# Concatenate the columns, handling empty strings, and remove extra spaces
df['Info'] = df[columns_to_concat].apply(lambda row: ', '.join(filter(None, row.astype(str))), axis=1)

In [8]:
# Concatenate the columns 'Market Cap' and 'AUM'
# Replace '-' with empty string in both columns

df['MktCap AUM'] = df['Market Cap'].replace('-', '') + df['AUM'].replace('-', '')

In [9]:
# Create new column by converting to numeric values in millions
df['MktCap AUM, M'] = df['MktCap AUM'].apply(convert_B_M_K_to_million)
df['Avg Volume, M'] = df['Avg Volume'].apply(convert_B_M_K_to_million)
df['Volume, M'] = (pd.to_numeric(df['Volume'].str.replace(',', ''), errors='coerce') / 1_000_000)

In [10]:
# Convert 'Beta', 'ATR', 'Rel Volume', and 'Price' to float
for col in ['Beta', 'ATR', 'RSI', 'Rel Volume', 'Price']:
  # Clean and convert to numeric
  df[col] = pd.to_numeric(
    df[col].str.replace('$', '').str.replace(',', ''),
    errors='coerce'  # Convert invalid values to NaN
  )

# Verify the conversion
for col in ['Beta', 'ATR', 'Rel Volume', 'Price']:
  print(f"{col}: {df[col].dtype}")

Beta: float64
ATR: float64
Rel Volume: float64
Price: float64


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   No.              1560 non-null   object 
 1   Ticker           1560 non-null   object 
 2   Company          1560 non-null   object 
 3   Sector           1560 non-null   object 
 4   Industry         1560 non-null   object 
 5   Market Cap       1560 non-null   object 
 6   Dividend %       1222 non-null   float64
 7   Perf Week %      1559 non-null   float64
 8   Perf Month %     1559 non-null   float64
 9   Perf Quart %     1555 non-null   float64
 10  Perf Half %      1552 non-null   float64
 11  Perf Year %      1543 non-null   float64
 12  Perf YTD %       1560 non-null   float64
 13  Beta             1541 non-null   float64
 14  ATR              1560 non-null   float64
 15  Volatility W %   1559 non-null   float64
 16  Volatility M %   1559 non-null   float64
 17  SMA20 %       

In [12]:
df.columns

Index(['No.', 'Ticker', 'Company', 'Sector', 'Industry', 'Market Cap',
       'Dividend %', 'Perf Week %', 'Perf Month %', 'Perf Quart %',
       'Perf Half %', 'Perf Year %', 'Perf YTD %', 'Beta', 'ATR',
       'Volatility W %', 'Volatility M %', 'SMA20 %', 'SMA50 %', 'SMA200 %',
       '50D High %', '50D Low %', '52W High %', '52W Low %', 'All-Time High %',
       'All-Time Low %', 'RSI', 'Gap %', 'Avg Volume', 'Rel Volume', 'Volume',
       'Price', 'Change %', 'Single Category', 'Asset Type', 'AUM', 'Info',
       'MktCap AUM', 'MktCap AUM, M', 'Avg Volume, M', 'Volume, M'],
      dtype='object')

In [13]:
my_cols = ['Ticker', 'Company', 'Info', 'MktCap AUM, M', 'Beta',
          'RSI', 'Perf YTD %', 'Perf Week %', 'Perf Month %', 'Perf Quart %', 'Perf Half %', 'Perf Year %',
          'SMA20 %', 'SMA50 %', 'SMA200 %',   
          '50D High %', '50D Low %', '52W High %', '52W Low %', 'All-Time High %', 'All-Time Low %',
          'ATR', 'Volatility W %', 'Volatility M %',  
          'Volume, M', 'Avg Volume, M','Rel Volume',
          'Price', 'Change %', 'Dividend %', 
          ]

In [14]:
# Create new df with my_cols and set Ticker as index
new_df = df[my_cols].set_index('Ticker')

# Sort by 'MktCap AUM, M' in descending order, with NaN values last
new_df = new_df.sort_values(by='MktCap AUM, M', ascending=False, na_position='last')

# Display info and first few rows to verify
display(new_df.info(), new_df)

<class 'pandas.core.frame.DataFrame'>
Index: 1560 entries, AAPL to SPGP
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1560 non-null   object 
 1   Info             1560 non-null   object 
 2   MktCap AUM, M    1560 non-null   float64
 3   Beta             1541 non-null   float64
 4   RSI              1559 non-null   float64
 5   Perf YTD %       1560 non-null   float64
 6   Perf Week %      1559 non-null   float64
 7   Perf Month %     1559 non-null   float64
 8   Perf Quart %     1555 non-null   float64
 9   Perf Half %      1552 non-null   float64
 10  Perf Year %      1543 non-null   float64
 11  SMA20 %          1560 non-null   float64
 12  SMA50 %          1560 non-null   float64
 13  SMA200 %         1560 non-null   float64
 14  50D High %       1560 non-null   float64
 15  50D Low %        1560 non-null   float64
 16  52W High %       1560 non-null   float64
 17  52W Low %       

None

,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,All-Time High %,All-Time Low %,ATR,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Dividend %
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",3336850.0,1.27,46.83,-11.30,0.63,-8.15,-14.24,-2.37,28.17,0.09,-3.53,-3.04,-11.15,6.58,-14.60,35.38,-14.60,349073.13,6.13,2.61,2.82,65.201326,52.26000,1.25,222.13,1.94,0.47
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2790640.0,0.99,35.51,-10.94,-4.50,-5.44,-14.32,-12.97,-10.92,-3.15,-7.28,-11.09,-16.28,-0.40,-19.85,-0.40,-19.85,471030.05,8.70,1.97,2.22,35.102847,22.59000,1.55,375.39,-0.90,0.87
NVDA,NVIDIA Corp,"Technology, Semiconductors",2644470.0,1.95,38.10,-19.29,-10.73,-13.24,-22.55,-12.62,20.09,-5.73,-12.90,-15.04,-27.25,3.45,-29.22,43.35,-29.22,325039.97,5.73,4.28,4.60,296.384596,277.24000,1.07,108.38,-1.18,0.04
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",2016320.0,1.38,35.42,-13.28,-6.40,-10.37,-16.20,-0.47,5.80,-4.00,-11.96,-4.97,-21.55,0.46,-21.55,25.49,-21.55,289820.00,6.47,2.88,3.18,63.001663,39.23000,1.61,190.26,-1.28,NaN
GOOGL,Alphabet Inc,"Communication Services, Internet Content & Inf...",1894590.0,1.03,33.26,-18.31,-7.78,-9.18,-20.94,-4.97,2.50,-6.44,-13.63,-11.55,-25.31,0.66,-25.31,5.04,-25.31,6339.78,5.00,2.99,2.94,54.193116,30.15000,1.80,154.64,0.20,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IYH,iShares U.S. Healthcare ETF,"Financial, Exchange Traded Fund, US Equities -...",3180.0,0.73,47.19,4.50,-1.22,-2.47,2.94,-5.83,-1.63,-0.54,-0.96,-2.14,-3.25,3.08,-8.56,6.12,-8.56,772.53,0.73,1.32,1.20,0.253784,0.32011,0.79,60.89,0.86,1.19
JAVA,JPMorgan Active Value ETF,"Financial, Exchange Traded Fund, US Equities -...",3180.0,0.83,45.21,0.38,-1.56,-3.77,-1.21,-0.08,6.40,-0.07,-2.40,-0.21,-5.05,3.02,-7.61,12.03,-7.61,38.97,0.85,1.28,1.37,0.248723,0.41967,0.59,63.54,0.84,1.46
IBDQ,iShares iBonds Dec 2025 Term Corporate ETF,"Financial, Exchange Traded Fund, Bonds - Corpo...",3180.0,0.13,60.21,0.40,0.00,0.00,0.52,0.28,1.41,0.12,0.13,0.50,-0.04,0.32,-0.04,1.90,-10.12,11.85,0.02,0.04,0.06,0.390708,0.56949,0.69,25.15,0.00,3.85


In [15]:
# Save to pickle file
new_df.to_pickle(dest_path)
print(f'save new_df to {dest_path}')

# # To load it later:
# loaded_df = pd.read_pickle('df_finviz.pkl')

save new_df to ..\data\df_finviz.pkl
